# Unit 2: Cleaning and Merging

## Contents

* [Getting Started](#Getting-Started)
* [Loading, Merging, and Cleaning Similar Data](#Loading,-Merging,-and-Cleaning-Similar-Data)
    * [Loading a CSV: Franklin County Audit Data](#Loading-a-CSV:-Franklin-County-Audit-Data)
    * [Parsing Errors: Licking County Auditor Data](#Parsing-Errors:-Licking-County-Auditor-Data)
    * [Extraction from a GIS Dataset: Fairfield County Auditor Data](#Extraction-from-a-GIS-Dataset:-Fairfield-County-Auditor-Data)
* [Joining Related Datasets](#Joining-Related-Datasets)
* [Lab Answers](#Lab-Answers)
* [Next Steps](#Next-Steps)
* [Resources and Further Reading](#Resources-and-Further-Reading)
* [Exercises](#Exercises)

### Lab Questions

[1](#Lab-1), [2](#Lab-2), [3](#Lab-3), [4](#Lab-4), [5](#Lab-5), [6](#Lab-6), [7](#Lab-7), [8](#Lab-8), [9](#Lab-9),  [10](#Lab-10), [11](#Lab-11), [12](#Lab-12), [13](#Lab-13), [14](#Lab-14), [15](#Lab-15)

## Getting Started

In addition to libraries we used in the last unit, this notebook relies on the [GeoPandas](library) to process data from a [geographic information system](https://en.wikipedia.org/wiki/Geographic_information_system).  To install the library, we can use `pip`.

If using [Anaconda](https://www.anaconda.com/download) and the following pip command fails, open the Anaconda prompt on your computer and run the following

```
conda install --yes geopandas
```

In [ ]:
import sys
!{sys.executable} -m pip install geopandas

We'll be working with county auditor data containing real estate information.  To reduce the memory and time required to process the data, the size of one of the datasets was reduced; the *sample_file()* function below contains the code to do this. Two arguments are required when the function is called, *input_file* and *output_file*, to specify the source and target files, respectively.  An keyword argument, *fraction*, can be specified to set the desired size of the output file relative to the input file; the default value is 0.1.  

First, *sample_file()* calls *get_line_count()* to calculate the total number of lines in the source file. The total is multiplied by the specified fraction and truncated to the nearest integer to determine the number of lines that should be in the output file.  Next, the [*sample()*](https://docs.python.org/3/library/random.html#random.sample) function from the random module is used to select a sampling of line numbers from a range of values starting at 1 and ending at the last line number in the source file; the number of line numbers in the sample is equal to the calculated sample line count.  Finally, the function iterates through the source file, line-by-line, and copies a line to the output file if that line's line number is in the sample of line numbers.

The *sample_file()* function was used to reduce the Franklin County Auditor data from over 400,000 lines to about 40,000 lines.

In [ ]:
import linecache
import random

def get_line_count(input_file):
    """Count number of lines in a file"""
    count = 0
    with open(input_file) as infile:
        for line in infile:
            count += 1
    return count

def sample_file(input_file, output_file, fraction=0.1):
    """Exctract a subset of lines from a file"""
    total_line_count = get_line_count(input_file)
    sample_line_count = int(fraction * total_line_count)  # fraction of total
    sample_line_numbers = random.sample(range(1, total_line_count), 
                                        sample_line_count)  # sample of line numbers
    sample_line_numbers.sort()
    sample_line_numbers.insert(0, 0)
    with open(output_file, 'w') as outfile:
        for line_number in sample_line_numbers:
            line = linecache.getline(input_file, line_number + 1)
            outfile.write(line)
                    

## Loading, Merging, and Cleaning Similar Data

Often, the analysis we would like to complete requires gather data from multiple sources.  Working with multiple sources can present some challenges that should be addressed before analyzing the data.  Rather than keeping source data separate, its usually convenient to store similar data together - combining separate sources into one database table to similar data store. Combining sources require extracting relevant data and reorganizing it to fit the target structure.  

In addition to simply extracting the data, we often need to address data quality issues as well; examples of quality issues include

- *duplication*: the dataset unnecessarily includes repeated data
- *inconsistency*: different values are used to represent the same thing or the values do not fit the defined schema
- *incompleteness*: data is missing from the dataset
- *inaccuracy*: the data does not reflect what it purports to measure or represent

Resolving duplication issues is usually straightforward: duplicate data is removed before conducting further analysis. Inconsistencies can be resolved by determining the appropriate values and transforming the data as needed; however, realizing that multiple values correspond to the same thing might require some examination of the data.  While it can be easy to detect incompleteness of data, the approach for resolving the issue might be context-specific. Should a default value be used? Should a randomly generated value within some range be substituted? Should records with missing data be dropped entirely? Inaccuracies tend to be more difficult to detect and to resolve as they often require examining how the source data was collected.  

In the following examples, we'll work on aggregating data from three different county auditor datasets. Our immediate objective is to collect any data regarding appraisal values, sale price, total area, the number of rooms, information about heating and cooling, and the year built for residential properties. Later, we'll use this data to determine if there's a relationship between price or appraisal value and the other properties.

### Loading a CSV: Franklin County Audit Data

The first data source from which we'll extract data is a CSV containing [data from the Franklin County Auditor](ftp://apps.franklincountyauditor.com/). As noted above, the data as been sampled to reduce its size from 400,000 records to 40,000.  The Auditor's site provides documentation of the dataset.  Though the documentation appears to be outdated, it does provide some useful information.

In [ ]:
# display auditor documentation in the notebook
from IPython.display import IFrame
IFrame("./data/02-franklin-description.pdf", 800, 600)

We can use the panda's *read_csv()* method to load the data.

In [ ]:
# load data
import pandas as pd
franklin = pd.read_csv('./data/02-franklin.csv')

With the data loaded, we can now begin examining it. To start, we can see the complete list of columns in the dataset.  Compare this to the columns in the documentation - there are column names in the dataset that do not appear in the documentation and column names in the documentation that do not appear in the dataset.

In [ ]:
franklin.columns.tolist()

We can view the first few rows using the DataFrame's *head()* method.  We'll increase the number of columns displayed to 200 to accommodate the datasets we'll be working with. 

In [ ]:
pd.set_option('display.max_column', 200)
franklin.head()

Examining these rows, we can get a sense of the type of data in each column.  We can also see that some values are `NaN` which stands for "Not a Number" and is used when no value is present, i.e. when data is missing.  We'll address these values later.

As noted earlier, we'd like to extract appraisal value, sale price, and other data for residential real estate.  Based on the documentation, the `PCLASS` field should indicate a parcel's property class.  The first few rows of data are consistent with the documentation.  To see all the values that appear in the `PCLASS` field, we can use the *unique()* method for that column.

In [ ]:
franklin.PCLASS.unique()

We can also see the number of records with each value using the *value_counts()* method.

In [ ]:
franklin.PCLASS.value_counts()

The documentation indicates that the `PROPTYP` also includes property type information; however, the first five rows do not have values for this field.  

<hr>
<a name="Lab-1"></a><mark> **Lab 1** Using *unique()*  or *value_counts()* in the cell below, confirm that none of the rows have data for the `PROPTYP` field.</mark>

<hr>
In a future unit, we'll explore how price or appraisal value is dependent on other factors such as number of bathrooms or the year in which a building a was built. In order to do this analysis, we'll need to extract the relevant data. 

Based on the documentation and the first few rows of data, we might be interested in extracting the following columns from the larger dataset.

- `APPRLND`: appraisal land value
- `APPRBLD`: appraisal building value
- `PCLASS`: property class
- `PRICE`: sales price
- `AREA_A`: building area
- `ROOMS`: total number of rooms
- `BATHS`: number of full bathrooms
- `HBATHS`: number of half bathrooms
- `BEDRMS`: number of bedrooms
- `AIRCOND`: heating and/or air conditioning
- `FIREPLC`: presence of a fireplace
- `YEARBLT`: the build year

To extract these columns, we'll first create a list containing their names then create a copy of the DataFrame consisting only of the columns.

In [ ]:
# home properties or fields we can use to filter out data
franklin_columns = ['APPRLND', 'APPRBLD', 'PCLASS', 'PRICE', 'AREA_A', 'ROOMS','BATHS', 
                    'HBATHS', 'BEDRMS', 'AIRCOND', 'FIREPLC','YEARBLT']

# copy vs view
franklin_subset = franklin[franklin_columns].copy()

The data in `franklin_subset` is a copy of the source data.  We can manipulate the copy while leaving the full dataset unchanged.  This can be helpful if we make a mistake or need to see what a value might have been prior to manipulation. Alternatively, we could just reload the data whenever necessary.

The first thing we can do is remove data for non-residential properties.  As shown above, 37,902 records correspond to residential properties.  To filter the data, we can use a mask and bracket notation with the DataFrame.  The mask we'll need is one that evaluates to `True` when the value of `PCLASS` is `R`.

One we've filtered the data, we can drop the `PCLASS` column as it is no longer needed.  To do this, we'll use the DataFrame's *drop()* method and specify the column name and axis.  We'll specify an axis value of *1* to indicate that we'd like to drop a column as opposed to a value of *0* to drop a row. We'll also use the *inplace* keyword argument to indicate that we'd like to manipulate the DataFrame itself rather than to return a DataFrame with the dropped column.

In [ ]:
# filter the data using a mask
franklin_subset = franklin_subset[franklin_subset.PCLASS == 'R']

# drop the PCLASS column
franklin_subset.drop(['PCLASS'], axis=1, inplace=True)

We can confirm that that the DataFrame has been filtered by comparing number of records in the `franklin_subset` DataFrame to the number of records in the `franklin` DataFrame.

<hr>
<a name="Lab-2"></a><mark> **Lab 2** In the cell below, use *len()* and a comparison operator to confirm that the number of records in the `franklin_subset` DataFrame is less than the number of records in the `franklin` DataFrames.</mark>

<hr>

Let's look at the the appraisal-related fields, `APPRLND` and `APPRBLD`.  From above, we know that that data in the `APPRBLD` field is stored as floating point numbers. We can use the *describe()* method to calculate some descriptive statistics for `APPRBLD`.

In [ ]:
franklin_subset.APPRBLD.describe()

Notice that the minimum value is zero.  Let's see how many records have a building appraisal value of zero. Because the data is stored as floating point numbers, we should be aware of the [issues](https://docs.python.org/3/tutorial/floatingpoint.html) related to floating point values.  If we choose to continue working with the data as floating point values, we can use the [NumPy *isclose()*](https://docs.scipy.org/doc/numpy/reference/generated/numpy.isclose.html) function to create a mask to compare values to zero.

In [ ]:
len(franklin_subset[pd.np.isclose(franklin_subset.APPRBLD, 0)])

As an alternative to working with floating point values, we can convert a column's datatype to `int` when appropriate.  Here, an integer would represent whole dollar amounts and would be meaningful; if decimals are used to record fractions of a dollar, we won't loose much information.  As a Series, each column has an *astype()* method that can be used to convert the column's type.  The method creates a copy so we have to reassign the DataFrame's column when doing the conversion.

In [ ]:
franklin_subset['APPRBLD'] = franklin_subset.APPRBLD.astype(int)

Now that the data is stored as integers, we can make comparisons more directly using the standard operators.

In [ ]:
len(franklin_subset[franklin_subset.APPRBLD == 0])

Let's filter the data to include only the rows where the building appraisal value is greater than zero.

In [ ]:
franklin_subset = franklin_subset[franklin_subset.APPRBLD > 0]

<hr>
<a name="Lab-3"></a><mark> **Lab 3** In the cell below, filter the `franklin_subset` DataFrame to exclude rows that have a `APPRLND` of zero.</mark>

<hr>

To simplify comparisons and other analysis later, we might choose to combine the data related to number of bathrooms into on column.  Because the data currently distinguishes between full and half baths, we can calculate the total number of bathrooms as the sum of the value of `BATH` and half the value of `HBATHS`.  Note that this has the effect of counting two half-bathrooms as a full bathroom; while two half-bathrooms might effect price differently than a full bathroom, we'll effectively ignore any such effect.

<hr>
<a name="Lab-4"></a><mark> **Lab 4** The data type of the `HBATH` and `HBATHS` should a numeric type (an integer or a floating point value) in order to calculate the combined value directly from the existing values.  In the cell below, use the `dtypes` property to confirm that the `HBATH` and `HBATHS` columns have a numeric data type.</mark>

<hr>

Rather than using a for loop and calculating net number of bathrooms for each row, pandas supports element-wise multiplication and addition allowing use to do the following.  For this calculation we will treat missing data in the same was as a zero value.  To do this, we use the *fillna()* method for the appropriate column and specify the value we'd like to use in place of missing values - zero, in this case.

In [ ]:
franklin_subset["Bathrooms"] = franklin_subset.BATHS.fillna(0) + 0.5 * franklin_subset.HBATHS.fillna(0)

This calculates the number of bathrooms as defined above and creates stores each row's value in a new column named `Bathrooms`.  

<hr>
<a name="Lab-5"></a><mark> **Lab 5** We no longer need the `BATHS` or `HBATHS` columns. In the cell below, use the *drop()* method to remove these columns from the `franklin_subset` DataFrame.</mark>

<hr>

Next, let's look at the `AIRCOND` column.  The documentation indicates that the field can take one of three values:

- 0: No heating or air conditioning
- 1: Heat
- 2: Air conditioning and heat

Let's compare this to the actual values in the dataset.

In [ ]:
# unique values in aircond
franklin_subset.AIRCOND.unique()

As we can see the `AIRCOND` column doesn't contain any records with a value of 2.  At this point, we might contact the person responsible for maintaining the data for clarification.  For our work, we'll that all residential parcels in the dataset have heat and `AIRCOND` here indicates whether or not air conditioning is available.  We can change the data type of the column to reflect this.  

In [ ]:
franklin_subset.AIRCOND = franklin_subset.AIRCOND.astype('bool')

Moving on to to the `FIREPLC` column, we can list the unique values to see that the dataset is again inconsistent with the documentation; rather than containing a single character representing the presence or absence of a fireplace the dataset instead contains integer values that likely indicate the number of fireplaces installed.

In [ ]:
franklin_subset.FIREPLC.unique()

We can see that `nan` is among the values.  

<hr>
<a name="Lab-6"></a><mark> **Lab 6** In the cell below, use the *fillna* property with the `FIREPLC` column to replace missing values with zero.  Either reassign the DataFrame's `FIREPLC` column with modified data or specify `inplace=True` as an argument to *fillna()* to alter the column in-place.</mark>

<hr>

At this point we have the following columns and data types.

In [ ]:
franklin_subset.dtypes

Before moving on to the next dataset, it might be useful to give the columns more descriptive names.  First, let's create a copy of the DataFrame in case we need access to the data in its current state later.

In [ ]:
home_data = franklin_subset.copy()

To rename the columns, we can use the DataFrame's *rename()* method. When calling the method, we can pass a dictionary that maps the existing column names to new names. We'll also specify that we want to change column names rather than index labels by specifying `axis=1` and that we'd like to alter the DataFrame itself rather than return a copy with the alteration using `inplace=True`.

In [ ]:
home_data.rename(
    {'APPRLND': 'AppraisedLand',
     'APPRBLD': 'AppraisedBuilding',
     'PRICE': 'SalePrice',
     'AREA_A': 'Area', 
     'ROOMS':'Rooms',
     'BEDRMS':'Bedrooms',
     'AIRCOND': 'AirConditioning', 
     'FIREPLC': 'Fireplaces', 
     'YEARBLT': 'YearBuilt'
    },
    axis=1 ,
    inplace=True
)

<hr>
<a name="Lab-7"></a><mark> **Lab 7** In the cell below, verify that the columns of the `home_data` DataFrame have been changed.</mark>

<hr>

As noted above, we'd also like to record whether or not a parcel includes heating.  Earlier we assumed that all residential properties in the data set did include heating.  To add a column with the same value for each row, we can write a statement that assigns that value to the new column in the DataFrame.  Similarly, we'll add a `County` column to indicate the source of the data.

In [ ]:
home_data['Heat'] = True
home_data['County'] = "Franklin"

We can view the first few rows to examine the state of our data before moving on to the next dataset.

In [ ]:
home_data.head()

### Parsing Errors: Licking County Auditor Data

We can augment the Franklin County Auditor data with data from the [Licking County Auditor](https://www.lickingcountyohio.us/).  The data we'll use was obtained directly from the auditor's site and was not sampled or modified.  Let's try loading the data stored in `data/02-licking.txt`.

In [ ]:
licking = pd.read_csv("./data/02-licking.txt")

The exception indicates that there was a problem parsing the data; specifically, pandas expected 33 fields but found 34 on line 3.  This could be due to pandas incorrectly guessing what the delimiter is.  We could use the csv module's *Sniffer* class to detect the delimiter but visual inspection will suffice.

In the code below, we'll print the first five lines.

In [ ]:
line_number = 0 
with open("./data/02-licking.txt") as infile:
    while line_number < 5:
        print(infile.readline())
        line_number += 1

Examining the output, we can see that the delimiter is probably a semicolon rather than a comma.  The pandas *read_csv()* method takes a keyword argument, *delimiter*, that will allow us to specify the appropriate value.

In [ ]:
licking = pd.read_csv("./data/02-licking.txt", delimiter=";")

The exception message indicates that pandas made it farther into the file before encountering an error.  On line 1608, it, pandas expected to find 181 fields based on the previous lines but instead found 182.  Let's investigate further.

While we could iterate through the file and collect the line or lines that are of interest to use, we can use the *linecache* module to access a specific line within a file.  The code below extracts a typical line (one that did not cause a parser error) and the line that causes a problem.  After extracting the lines, the code displays their content.

In [ ]:
import linecache
typical_line = linecache.getline("./data/02-licking.txt", 2)
error_line = linecache.getline("./data/02-licking.txt", 1608)

display(typical_line)
display(error_line)

Printing the lines in their entirety isn't very revealing.  

<hr>
<a name="Lab-8"></a><mark> **Lab 8** A difference in the number of fields could be caused by a difference in the number of delimiters. In the cell below use the [*count()*](https://docs.python.org/3/library/stdtypes.html#str.count) method with each line to display the number of times the delimiter appears.</mark>

<hr>

It would be helpful if we could compare each fields values between the two lines.  To do this we'll use the String [*split()*](https://docs.python.org/3/library/stdtypes.html#str.split) method to separate each line into a list of field values.  In addition to the two lines we already have, we'll retrieve the first line from the data for column names.  We can use the itertools module's [zip_longest](https://docs.python.org/3/library/itertools.html#itertools.zip_longest) function to combine the list of values extracted from each line for comparison.  

In [ ]:
from itertools import zip_longest

header_line = linecache.getline("./data/02-licking.txt", 1)

header_entries = header_line.split(";")
typical_entries = typical_line.split(";")
error_entries = error_line.split(";")

for entry in zip_longest(header_entries, typical_entries, error_entries):
    print(entry)

Compare the values for the `fldTopo` header.  The "typical line" has no value whereas the "error line" has a value that seems related to the value associated with the previous field, `fldLUC`.  If we look back to the the display of each line's content, we can see that "430 Resturant" and "cafeteria and/or bar" are separated by a semicolon but should be kept together rather than split apart as different field values; note that "Restaurant" is misspelled in the source data.  The source data should use quoting if a delimiter appears as part of a data value or avoid using the delimiter in such a capacity.

Now that we know what the problem is, there are a variety of ways to address the problem.  One way is to replace all instances of "430 Restaurant; cafeteria and/or bar" in the source text with something that doesn't have a semicolon prior to loading it in pandas.  In the code below, we assign the problematic value and its replacement value to variables.  After reading the content of the file, we use the *replace()* method to substitute occurrences of the first value with the second. We then load the data into pandas.  Because the pandas *read_csv()* function is expecting a file or stream, and not a string or bytes, we use the [*StringIO*](https://docs.python.org/3/library/io.html#io.StringIO) class to create a stream from the altered content. We specify "python" as the *engine* in the *read_csv()* method to avoid warnings about memory.  The Python CSV engine provided by pandas is more feature-complete but is slower than the default C engine.

In [ ]:
import io 
old_value = "430 Resturant; cafteria and/or bar"
new_value = "430 Resturant, cafeteria and/or bar"

with open("./data/02-licking.txt") as infile:
    content = infile.read()
    
content = content.replace(old_value, new_value)
    
licking = pd.read_csv(io.StringIO(content), delimiter=";", engine="python")

While this was relatively straightforward, there are disadvantages to this method.  The primary disadvantage here is that we iterate through the content of the file several times: first we read all the content, then we iterate through it to find and replace the problematic value, then iterate through it to load it into pandas; usually we only iterate through the file once when loading it into pandas.  While this is fine for relatively small files, we should avoid looping through the entirety of a file whenever possible.

An alternative method would be to make use of pandas' support for [regular expressions](https://docs.python.org/3.2/library/re.html) when specifying the delimiter. We can use a [negative look-behind assertion](https://www.regular-expressions.info/lookaround.html) to indicate that a delimiter is any semicolon that isn't immediately preceded by the string "Resturant".  We could do this with the following call to *read_csv()*:

```python
licking = pd.read_csv("./data/02-licking.txt", delimiter="(?<!Resturant);", engine="python")
```

With the data loaded, let's display the first few lines to get sense of the data.

In [ ]:
licking.head()

As with the Franklin country dataset, we'd like to filter this dataset for only residential buildings.  Unfortunately, there isn't documentation available to describe the content of each column so we'll have to do our best to infer meaning from the column name and values. Looking at the data above, it looks like `fldPropertyType` or `fldStyle` might be useful to determine which properties are residential and which are not.

In [ ]:
licking.fldPropertyType.unique()

In [ ]:
licking.fldStyle.unique()

It looks like most of style values are related to residential-type properties.  At this point, we might decide to choose specific styles to filter on or choose to simply exclude records without style information or those that correspond to a commercial style.  

Let's see the styles associated with the *Dwelling* property type.  

In [ ]:
licking[licking.fldPropertyType == 'Dwelling'].fldStyle.unique()

Filter the data to include only the *Dwelling* property didn't reduce the number of styles.  For this example, we'll filter the data to include only *Single Family*, *MFD Home*, *Tri-Level*, *Duplex*, *Bi-Level*, *Multi-Level*, *Condominum*, *Mobile Home*, *Triplex*, and *4-Level*.  Note that *Condominum* is misspelled in the source data.

We can create a list of acceptable style values now that can be used to filter the data later.

In [ ]:
licking_styles = ['Single Family', 'MFD Home', 'Tri-Level', 'Duplex',
                     'Bi-Level', 'Multi-Level', 'Condominum', 'Mobile Home',
                     'Triplex', '4-Level']

Let's consider the other columns we'll need.  We had collected sales price data from the Franklin county dataset.  In this dataset, there are quite a few columns with "sales" in the name.

In [ ]:
[column for column in licking.columns if "sales" in column.lower()]

Looking at the sample data above, we will likely be interested in the collection of `fldSalesPrice` columns to determine the sales price. Let's look at the values of these columns for a small number of rows.

In [ ]:
licking[['fldSalesPrice1', 'fldSalesPrice2', 'fldSalesPrice3', 'fldSalesPrice4']].head(10)

We have nonzero, zero and `NaN` values.  In addition to these columns, the data also contains `fldSalesDate` columns. To get a better idea of what the prices represent, let's look at the date columns as well.

In [ ]:
licking[['fldSalesPrice1', 'fldSalesPrice2', 'fldSalesPrice3', 'fldSalesPrice4', 
         'fldSalesDate1', 'fldSalesDate2', 'fldSalesDate3', 'fldSalesDate4']].head(10)

As we move from the first price/date column to the second, the second price/date column to the third, and so on, we move backward in time.  It seems reasonable then that `fldSalesPrice1` represents the most recent sales price and the other columns are used to record historic sales data (if it exists).  We'll use the most recent sales price for our work so we'll only need `fldSalesPrice1`.

Just as with the word "sale", there are a number of columns that contain the word "area". 

<hr>
<a name="Lab-9"></a><mark> **Lab 9** In the cell below, display all the columns with "area" in their name.</mark>

<hr>

Here, we'll assume `fldFinishedLivingArea` contains the data need for area.

Next, lets look for bathroom data.

In [ ]:
[column for column in licking.columns if "bath" in column.lower()]

We have columns corresponding to both full and half bathrooms as before but there is a third column for "other".  Let's see what values for this field look like.

In [ ]:
licking.fldOtherBaths.value_counts()

The values themselves don't give a clear idea of what the field represents.  Given the lack of documentation, we'd likely contact the person or group responsible for the data for clarification; for our work here, we'll assume this field corresponds to quarter bathrooms.  

Examining the sample data above, we can identify the other columns of interest.  Specifically, we'll extract the following columns from the Licking Country dataset.

In [ ]:
#home_columns = ["AppraisedLand", "AppraisedBuilding", "LastSalePrice", "Area", "Rooms", "Bedrooms", "Bathrooms", "AirConditioning", "Heat", "Fireplaces", "YearBuilt" ]
licking_columns = ["fldMarketLand", "fldMarketImprov", "fldSalesPrice1", "fldFinishedLivingArea", "fldRooms", "fldBedrooms", "fldFullBaths", "fldHalfBaths", "fldOtherBaths",  "fldHeating", "fldCooling", "fldFireplaceOpenings", "fldYearBuilt"]

We can create a mask to filter the data based on style values.  Rather than compare one value to another as we did when filtering the Franklin Country data, we'll instead check if a values is among a list of values.  To do this, we can us the column's *isin()* method to test a value's membership in a specified list. 

Below we check if each value in the `fldStyle` column is in the `licking_styles` list we created earlier.

In [ ]:
licking.fldStyle.isin(licking_styles)

We can apply this mask in the usual way using bracket notation.

In [ ]:
licking_subset = licking[licking.fldStyle.isin(licking_styles)].copy()

We can confirm that the filtered data contains only the style values we had wanted.

In [ ]:
licking_subset.fldStyle.unique()

Now, let's extract only the columns we want. We use bracket notation again with the list of columns we specified above.

In [ ]:
licking_subset = licking_subset[licking_columns]

We can display the first few rows of `licking_subset` to confirm we've extracted what we wanted.

In [ ]:
licking_subset.head()

We can combine the various bathroom columns into one `Bathroom` column in the same way we combined them for the Franklin County dataset.

<hr>
<a name="Lab-10"></a><mark> **Lab 10** In the cell below, combine the values for full baths, half baths and other baths into one columns named `Bathrooom`.  Assume the value in `fldOtherBaths` is equivalent to a quarter of a full bathroom.  
    
Additionally, drop the original bathroom-related columns after computing the values for the new column
</mark>

<hr>

Let's look at heating and cooling data . We extracted two columns from the original dataset `fldHeating` and `fldCooling` that contain heating and cooling data, respectively.  Let's look at the heating data first.

In [ ]:
licking_subset.fldHeating.value_counts()

The target dataset doesn't differentiate among different data sources - it only indicates whether the property has heating or not.  For the Licking County data, we'd like to associate `False` with `No Heat` and `True` otherwise. We can do this by comparing values.

In [ ]:
licking_subset.fldHeating = licking_subset.fldHeating != "No Heat"

We can calculate the value counts of the field to confirm that the number of `False` entries corresponds to the previous number of `No Heat` entries.

In [ ]:
licking_subset.fldHeating.value_counts()

<hr>
<a name="Lab-11"></a><mark> **Lab 11** In the cell below, replace the values in the `fldCooling` column with `True` to indicate that a property has cooling and `False` otherwise.

</mark>

<hr>

We can assume `fldFireplaceOpenings` correspond to fireplaces.  The only change we'll make is is to replace missing data with zeros.

In [ ]:
licking_subset.fldFireplaceOpenings.fillna(0, inplace=True)

That should be the last modification needed for the Licking County data.  In order to combine the `home_data` and `licking_subset` DataFrames, we need to make sure they have the same column names.  We'll rename columns in the same way we did previously.

In [ ]:
licking_subset.rename(
    {'fldMarketLand': 'AppraisedLand',
     'fldMarketImprov': 'AppraisedBuilding',
     'fldSalesPrice1': 'SalePrice',
     'fldFinishedLivingArea': 'Area', 
     'fldRooms':'Rooms',
     'fldBedrooms':'Bedrooms',
     'fldHeating': "Heat",
     'fldCooling': 'AirConditioning', 
     'fldFireplaceOpenings': 'Fireplaces', 
     'fldYearBuilt': 'YearBuilt'
    },
    axis=1 ,
    inplace=True
)

We can also add a column to the identify the source of this data.

In [ ]:
licking_subset['County'] = "Licking"

We can confirm that the columns in `home_data` and `licking_subset` are the same. Any differences will have to be corrected before merging the data. 

In [ ]:
home_data.columns

In [ ]:
licking_subset.columns

To see how we can combine the DataFrames, let look at an example.  We'll start with two DataFrames, each wit columns `A` and `B` and with two rows.

In [ ]:
d1 = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
d2 = pd.DataFrame([[5, 6], [7, 8]], columns=list('BA'))

display(d1)
display(d2)

To append the content of one DataFrame to the end of another, we can use the DataFrame *append()* method.  We specify `ignore_index=True` to prevent duplication of index labels.

In [ ]:
d1.append(d2, ignore_index=True)

Note that the *append()* method does not modify the original DataFrames directly but instead returns the combined DataFrame.  We can append the `licking_subset` DataFrame to `home_data` and assign the result to `home_data`.

In [ ]:
home_data = home_data.append(licking_subset, ignore_index=True)

We can see that `home_data` now has data for two counties.

In [ ]:
home_data.County.value_counts()

### Extraction from a GIS Dataset: Fairfield County Auditor Data

The final dataset we'll work with is the [Fairfield County Auditor Data](https://www.co.fairfield.oh.us/gis/).  This data is stored as [GIS](https://en.wikipedia.org/wiki/Geographic_information_system) data so loading it won't be as straightforward as reading a text file.  To access the data, we'll use the [GeoPandas](http://geopandas.org/) library, which will load the GIS data into a DataFrame so we can work with it in the same way we manipulated the other datasets. Recall that we installed the library using pip at the beginning of this notebook; with the libary installed, we can import it.

In [ ]:
import geopandas

The GIS data we're working with is stored in a [format](http://doc.arcgis.com/en/arcgis-online/reference/shapefiles.htm) specified by [ESRI](https://www.esri.com/en-us/home), the developer of [ArcGIS](https://www.arcgis.com/features/index.html), a popular GIS software product.  Data in this format is stored across several files and can be distributed as a single zip file.  We can load the data from the zip file using GeoPanda's *read_file()* function.  The data we'll be using is stored in `data/02-fairfield-gis.zip`.

In [ ]:
#https://www.co.fairfield.oh.us/gis/
fairfield = geopandas.read_file("zip://data/02-fairfield-gis.zip")

The object returned by the *read_file()* method is a [GeoDataFrame](http://geopandas.org/data_structures.html#geodataframe), an extension of the pandas DataFrame with additional functionality.  The attributes and methods we've used with other DataFrames are available to use when working with GeoDataFrames.  For example, we can see the first few rows in the Fairfield County data using the *head()* method.

In [ ]:
fairfield.head()

For the most part, this looks like what we'd expect for county auditor data. The last column, however, is something we haven't seen yet.  The `geometry` column contains [data](http://desktop.arcgis.com/en/arcmap/10.3/analyze/arcpy-classes/geometry.htm) used to represent the location and shape of geometric features.  We can use this data to construct plots with map data.  

In the code below, we use the [Matplotlib](https://matplotlib.org/) library to create a plot; we'll work with this library again later.  Because the geometric data represents geographic objects on Earth's surface, position information is stored using a [coordinate reference system](http://geopandas.org/projections.html).  For simpler manipulation, the code below converts data to use a reference system that relies on standard latitude and longitude which can be used in masks to filter the data.  Once filtered, the data is plotted.  In the resulting plot of [Lancaster](https://www.google.com/maps/place/Lancaster,+OH+43130/@39.7234464,-82.678719,12z/data=!3m1!4b1!4m5!3m4!1s0x88478a5e4f80f267:0x136dd5d79e3b4de5!8m2!3d39.7136754!4d-82.5993294), we can see features such as roads.

In [ ]:
fairfield.crs

In [ ]:
# lancaster
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (12, 10)
fairfield = fairfield.to_crs({'init': 'epsg:4326'})
fairfield[(fairfield.geometry.centroid.x >= -82.7) &
          (fairfield.geometry.centroid.x <= -82.5) &
          (fairfield.geometry.centroid.y >= 39.7) &
          (fairfield.geometry.centroid.y <= 39.75)].plot()


Returning to the task at hand, let's work on cleaning/filtering the Fairfield County data and merging it with the existing data.  

Access to data that was used to construct the GIS dataset is available through a link on the Fairfield County Auditors site. The data is hosted on an external [site](http://downloads.ddti.net/fairfieldoh/) and includes a description of the database structure.  We can load the documentation in the notebook; the description for dwelling data is given on page 13.

In [ ]:
IFrame("./data/02-fairfield-description.pdf#page=13", 800, 600)

Based on the first few rows and the documentation, we might be able to filter the data based on the values in the `CLASS` column.  Let's look at its values.

In [ ]:
fairfield.CLASS.value_counts()

We'll assume `R` represents residential data. We can see that we'll likely need the following columns as well.

- `SFLA`: Living area
- `YRBLT`: Year built
- `RMTOT`: Total rooms
- `RMBED`: Bedrooms
- `FIXBATH`: Bathrooms
- `FIXHALF`: Half-bathrooms
- `HEAT`: Heat code
- `PRICE`: Sales price
- `APRLAND`: Appraised land value
- `APRBLDG`: Appraised building value

We can filter the data and extract the columns of interest.

In [ ]:
fairfield_columns = ['SFLA', 'YRBLT', 'RMTOT', 'RMBED', 'FIXBATH', 
                     'FIXHALF', 'HEAT', 'PRICE', 'APRLAND',  'APRBLDG']
fairfield_subset = fairfield[fairfield.CLASS == 'R'][fairfield_columns].copy()

Let's look the first few rows of the DataFrame.

In [ ]:
fairfield_subset.head()

We can combine the `FIXBATH` and `FIXHALF` columns into a single column using the same method we used for the Franklin County data.

In [ ]:
fairfield_subset['Bathrooms'] = fairfield_subset.FIXBATH + 0.5 * fairfield_subset.FIXHALF
fairfield_subset.drop(["FIXBATH", "FIXHALF"], axis=1, inplace=True)

Turing to the `HEAT` column, the data documentation indicates that the values in this column represent a "heat code".

In [ ]:
fairfield_subset.HEAT.value_counts()

Among the tables in the database available online is one that defines these values.  The heat codes are as follows:

- 1: None
- 2: Basic
- 3: Air conditioning
- 4: Heat Pump

We'll have to extract both heating and cooling data from this column. Notice that the output of *value_counts()* includes a row count for what appears to be missing data.  Before continuing, let's try to determine why there is a missing value.  To begin, let's use the *unique()* method for a better representation of the distinct values.

In [ ]:
fairfield_subset.HEAT.unique() 

The missing value is an empty string.  At this point we need to decide if should assume that a missing value means no heat or some other type of heating that doesn't correspond to a code.  Let's look at a few rows where `HEAT` is an empty string.

<hr>
<a name="Lab-12"></a><mark> **Lab 12** Using a mask and the *head()* method, display the first five rows of `fairfield_subset` where the `HEAT` column has an empty string for a value.
</mark>

<hr>

It appears that records where `HEAT` is an empty string, correspond to parcels with no living area.  At this point we can filter the data again to exclude records with an empty string in `HEAT`.

In [ ]:
fairfield_subset = fairfield_subset[fairfield_subset.HEAT != '']

This leaves the following values in the `HEAT` column.

In [ ]:
fairfield_subset.HEAT.value_counts()

While we could write code that iterate through the rows of the DataFrame and sets heating and cooling values at the same time, it is easier to split this into two tasks: set the cooling value then set the heating value. 

We can create a new column, `AirConditioning` based on whether or not `HEAT` has a value of `'3'`. Because the column contains strings, it's important that our masks compare the column's values to another string rather than an iteger, i.e, our mask for air conditioning should be

```python
fairfield_subset.HEAT == '3'
```

rather than

```python
fairfield_subset.HEAT == 3
```

In [ ]:
fairfield_subset['AirConditioning'] = fairfield_subset.HEAT == '3'
fairfield_subset.AirConditioning.value_counts()

Similarly, we can assign a new value to `HEAT` based on the existing value. 

In [ ]:
fairfield_subset.HEAT = fairfield_subset.HEAT != '1'
fairfield_subset.HEAT.value_counts()

Let's see what the data looks like.

In [ ]:
fairfield_subset.head()

The final steps are to rename the columns, add data about the source, and append the Fairfield subset to our larger dataset.


<hr>
<a name="Lab-13"></a><mark> **Lab 13** In the cell below, rename the columns of the `fairfield_subset` DataFrame so they are consistent with the columns in `home_data`.
</mark>

<hr>

That leaves adding the county name and appending the data.

In [ ]:
fairfield_subset['County'] = 'Fairfield'
home_data = home_data.append(fairfield_subset, ignore_index=True)

We can see that our dataset contains data from three counties.

In [ ]:
home_data.County.value_counts()

Often when we work with data, we encounter duplication - repetition of data.  We can see if `home_data` contains duplicate data by comparing the number of rows that would be left if we removed duplicates using the *drop_duplicates()* method to the number of rows in the current DataFrame.

In [ ]:
len(home_data.drop_duplicates())/len(home_data)

This indicates that a little over 1% of our data corresponds to duplicates.  While the original data might not have contained duplicates, we created what appear to be duplicates by removing unneeded columns.  To see this more clearly, consider the following DataFrame.

In [ ]:
df = d1 = pd.DataFrame([[1, 2, 3], [1, 2, 4]], columns=list('ABC'))
df

The two rows of data are distinct.  However, if decide we no longer need column `C`, the rows will appear to be duplicates.

In [ ]:
df.drop(['C'], axis=1, inplace=True)
df

We can calculate the percentage of rows that would be left after removing duplicates as we did above.

In [ ]:
len(df.drop_duplicates())/len(df)

While the rows in `home_data` might have corresponded to distinct properties to begin with, at this point there is no way to distinguish between duplicated rows.  For now, however, we will leave the duplicates in the data knowing that they do represent different properties. 

Let's look at the data types of our columns.

In [ ]:
home_data.dtypes

Notice that `Area` data is stored as objects.  We would probably like to store area as an integer or as a floating point value.  Let's try to convert the area values to integers.

In [ ]:
home_data.Area = home_data.Area.astype(int)

The exception message indicates that some of the values contain commas. Before continuing, note the following.

In [ ]:
from collections import defaultdict

types = defaultdict(int)
for value in home_data.Area:
    value_type = type(value)
    types[value_type] += 1

types

The `Area` data contains some data stored as integers and other data stored as strings.  The number of strings corresponds to the number of entries from Licking country so the area data was likely stored with commas in that dataset.  To resolve this we can iterate through each row and, if the data is a string, remove any commas and convert to an integer. To iterate through the rows of a DataFrame we can use *iterrows()*.

In [ ]:
for index, row in home_data.iterrows():
    if isinstance(row.Area, str):
        new_value = int(row.Area.replace(",", ''))
        home_data.loc[index, 'Area'] = new_value

Iterating through the DataFrame row by row can be slow and should generally be avoided.  An alternative approach is to create function that would handle one value at a time and use the DataFrame's [*apply()*](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html) method.

Let's look at the data types for each column now.

In [ ]:
home_data.dtypes

As we loaded each data set, we checked some columns for missing values, represented by `NaN` by examining the value counts of those columns. We can check our `home_data` data frame for missing values we might have overlooked . In the code below, we first use the `isna()` method to return `True` for every value that is `NaN` and `False` otherwise.  We then calculate the sum of `True` values for each column using the `sum()` method. 

In [ ]:
home_data.isna().sum()

There are several columns with missing values.  For most of the columns, we might choose to remove any rows that contain missing information; for example, if we plan to use the data to see how different factors affect sales price, we probably don't want to keep records missing price data.  Before we start dropping rows, let's address the `Fireplaces` column - there are a significant number of missing values.  Recall that when we were working with the Fairfield data, we didn't have any fireplace data.  We can confirm that all the missing `Fireplaces` values correspond to records from Fairfield county.

In [ ]:
home_data[home_data.Fireplaces.isna()].County.value_counts()

We first filter the dataset to consist of only those rows where there are missing fireplace values then calculate the value counts for the `County` column.  Indeed, the missing values are entirely from the `Fairfield` dataset.  What we do next is dependent on what we want to do with the data.  For now, we'll leave those missing values and, if we later try to determine a relationship between the number of fireplaces and sales price, we'll have to account for the fact that over 40,000 rows are missing fireplace data.

For the other columns with missing values, we'll remove any rows with missing data.

In [ ]:
# iterate over the columns
for column in home_data.columns:
    # ignore the Fireplaces column
    if column == "Fireplaces":
        continue
    # filter the dataframe to include non-NaN values for the column
    home_data = home_data[home_data[column].notna()]

Looking at the number of missing values in each column shows that only `Fireplaces` is missing values.

In [ ]:
home_data.isna().sum()

We can use the `shape` property to confirm that we haven't accidentally delete most of the data.

In [ ]:
home_data.shape

We'll store the data in a SQLite database using SQLAlchemy and the DataFrame's [*to_sql()*](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) method; the first argument to this method specifies the table name we'd like to use an the third argument indicates that we would like to replace any existing data. 

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///data/output.sqlite')
home_data.to_sql("home_data", con=engine, if_exists='replace')

## Joining Related Datasets

In the previous examples, we worked on appending the rows of one DataFrame to another. Pandas supports a [variety of methods](https://pandas.pydata.org/pandas-docs/stable/merging.html) of combining data. Another common method is similar to a [database join](https://en.wikipedia.org/wiki/Join_%28SQL%29) where we combine combine the columns of two or more datasets.  Pandas DataFrames provide two methods that can be used for "joins": *join()* and *merge()*.  The *join()* method can be used when combining datasets based on index values and the *merge()* method can be used to combine datasets on any column values as well as index values; *merge()* is the more general method and *join()* ultimately relies on *merge()* to combine DataFrames.

In the next example, we'll load two datasets related to vehicles. The first is [EPA/Department of Energy Data tracking](https://www.fueleconomy.gov/) the the fuel economy of vehicles and the second is [Norwegian vehicle sales data](https://www.kaggle.com/dmi3kno/newcarsalesnorway).  In a future less, we'll explore relationships within and between these datasets but we can combine the datasets first. We begin by loading the EPA data.

<hr>
<a name="Lab-14"></a><mark> **Lab 14** In the cell below, use the Pandas *read_csv()* function to load the data from `./data/02-vehicles.csv` an store it in a variable named `epa_data`. You might encounter a warning message about columns containing mixed types. One solution is to use the more feature-complete Python engine rather than the faster C engine; to to this, add `engine='python'` as an argument to *read_csv()*.
</mark>

<hr>

An extract of the data description is presented below. 

In [ ]:
from IPython.display import HTML
HTML(filename="./data/02-vehicles-description.html")

As we typically do after loading a new dataset, let's look at the first few rows to get a sense of the data contained in the dataset.

In [ ]:
epa_data.head()

We'll explore the data futher later.  For now, the important columns to consider will be `make`, `model`, and `year`.

The next dataset contains data about Norwegian car sales and is stored using the [ISO 8859-1](https://en.wikipedia.org/wiki/ISO/IEC_8859-1) encoding rather than standard ASCII or UTF-8; as such, we must specify the encoding as a parameter to the *read_csv()* function.  After loading it, we can examine the first few rows.

In [ ]:
sales = pd.read_csv("./data/02-vehicle-sales-norway.csv", encoding="ISO-8859-1")
sales.head()

The dataset contains year, make, and model information but the `Model` column is the combination of make and model.  In order to combine the datasets, we'll need to separate these two pieces of data.  To start, we'll rename the current `Model` column to `MakeModel`; this will allow us to preserve the existing data. 

In [ ]:
sales.rename({"Model": "MakeModel"}, axis=1, inplace=True)

Looking at the existing model data, it looks like the value that appears in the `Make` column is repeated as the beginning of the `MakeModel` column.  We use the `replace()` method to replace the `Make` value with an empty string - effectively removing it.  We will then use the `strip()` method to remove any leading or trailing white space.  

We could iterate through the rows of the DataFrame using a for-loop; however, this tends to be slow. An alternative method is to use the DataFrame's *apply()* method that allows us to specify a function to [vectorize](https://en.wikipedia.org/wiki/Array_programming) an operation to an entire row or column.

In the code below, we first define the function that we would like to apply.  The function is written as though it is applied one row at a time - pandas handles the vectorization for us.  To use the function to alter the data, we use the *apply ()* method and specify the name of the function and that we would like to apply it to each row by specifying `axis=1` which indicates that we would like to apply it along multiple columns and allow us to access their values.   

In [ ]:
def remove_make(row):
    make = row['Make']
    make_model = row['MakeModel']
    # remove make and and leading/trailing white space
    return make_model.replace(make, "").strip()
    
sales["Model"] = sales.apply(remove_make, axis=1)

In [ ]:
sales.head()

It looks like that worked.  In order to join the two DataFrames, we'll specify the columns whose values will be compared for matching.  To simplify this, it is helpful to use the sames column names, including the same case, in both datasets. For our data, we can convert the columns in the sales data to lowercase. 

In [ ]:
sales.rename({col: col.lower() for col in sales.columns},
             axis=1, inplace=True)

To merge the data, we can use the *merge()* method of one of the two DataFrames.  When using *merge()*, we need to specify the other DataFrame and the columns used for matching.

In [ ]:
epa_sales = epa_data.merge(sales, on=["year", "make", "model"])
epa_sales.head()

There appears to be a problem.  For some reason, pandas wasn't able to find any matches for a give (year, make, model) value in the `epa_data` set with the data in the `sales` dataset.  Let's look at a sample of value for each dataset. We'll use the `values` property to see how the data is stored rather than show the nicely formated representation.

In [ ]:
epa_data[['year', 'make', 'model']].head().values

In [ ]:
sales[['year', 'make', 'model']].head().values

It looks like the values in the `make` column in the `sales` data have some trailing white space whereas the values in the `epa_data` do not.  This difference is enough to prevent pandas from matching the values.  To address this, we can create a function that strips white space from values.  To be safe, we can also convert the values to lowercase to ensure differences in case don't cause problems.

In [ ]:
# strip white space and convert to lower case
def strip_lower(x):
    return x.strip().lower()

Compare this function to the `remove_make()` function we defined above.  This function operates on an individual value rather than an entire row.  To apply this to the values in a column of a DataFrame, we should use the *apply()* method associated with the column itself rather than the DataFrame.  The following code applies the function to `make` and `model` columns of both DataFrames; the `year` data in both DataFrames are stored as integers and do not require this transformation.

In [ ]:
epa_data.make = epa_data.make.apply(strip_lower)
epa_data.model = epa_data.model.apply(strip_lower)

sales.make = sales.make.apply(strip_lower)
sales.model = sales.model.apply(strip_lower)

Let's check the values in the DataFrames again.

In [ ]:
epa_data[['year', 'make', 'model']].head().values

In [ ]:
sales[['year', 'make', 'model']].head().values

It looks like the white space has been removed and the make and model data are lowercased.  Let's try merging the DataFrames again.

In [ ]:
epa_sales = epa_data.merge(sales, on=["year", "make", "model"])
epa_sales.head()

It looks like the merge was successful.  We can use the DataFrame's *shape* property to see the number of columns and rows it contains.

In [ ]:
epa_sales.shape

The `epa_sales` DataFrame contains 1562 rows and 87 columns. We don't need to keep all these columns so let's select a subset.  

In [ ]:
epa_sales = epa_sales[['city08', 'co2', 'comb08', 'cylinders',
                       'displ', 'fuelType1', 'highway08', 
                       'make','model','VClass','year', 'quantity']].copy()

Let's save this DataFrame to the database use created previously so we can access the data later.

<hr>
<a name="Lab-15"></a><mark> **Lab 15** In the cell below, use the `epa_sales` DataFrame's *to_sql()* method to save the data in the same database that we stored property information. Use the table name `epa_sales`.
</mark>

<hr>

## Lab Answers

1.  ```python
    franklin.PROPTYP.unique()
    ``` 

    or 

    ```python
    franklin.PROPTYP.value_counts()
    ```
    
2. ```python
   len(franklin_subset) < len(franklin)
   ```
   
3. ```python
   franklin_subset['APPRLND'] = franklin_subset.APPRLND.astype(int)
   franklin_subset = franklin_subset[franklin_subset.APPRLND > 0]
   ```
   
4. ```python
   franklin_subset[['BATHS', 'HBATHS']].dtypes
   ```
   
5. ```python
   franklin_subset.drop(['BATHS', "HBATHS"], axis=1, inplace=True)
   ```
   
6. ```python
   franklin_subset.FIREPLC = franklin_subset.FIREPLC.fillna(value=0)
   ```
   
   or
   
   ```python
   franklin_subset.FIREPLC.fillna(value=0, inplace=True)
   ```
   
7. ```python
   home_data.columns
   ```
   
8. ```python
   display(typical_line.count(";"))
   display(error_line.count(";"))
   ```
   
9. ```python
   for column in licking.columns:
       if "area" in column.lower():
           display(column)
   ```
   
10. ```python
    licking_subset['Bathrooms'] = (licking_subset.fldFullBaths.fillna(0) + 
                                   0.5 * licking_subset.fldHalfBaths.fillna(0) + 
                                   0.25 * licking_subset.fldOtherBaths.fillna(0))
   licking_subset.drop(["fldFullBaths", "fldHalfBaths", "fldOtherBaths"], axis=1, inplace=True)
    ```
    
11. ```python
    licking_subset.fldCooling = licking_subset.fldCooling == "Central"
    ```
    
12. ```python
    fairfield_subset[fairfield_subset.HEAT == ''].head()
    ```
    
13. ```python
    fairfield_subset.rename(
        {'APRLAND': 'AppraisedLand',
         'APRBLDG': 'AppraisedBuilding',
         'PRICE': 'SalePrice',
         'SFLA': 'Area', 
         'RMTOT':'Rooms',
         'RMBED':'Bedrooms',
         'HEAT': 'Heat',
         'YRBLT': 'YearBuilt'
        },
        axis=1 ,
        inplace=True
    )
    ```

14. ```python
    epa_data = pd.read_csv("./data/02-vehicles.csv")
    ```
    
    or 
    
    ```python
    epa_data = pd.read_csv("./data/02-vehicles.csv", engine="python")
    ```

15. ```python
    epa_sales.to_sql("epa_sales", con=engine, if_exists='replace')
    ```

## Next Steps

Now that we've loading and cleansed the data to some extent, next step might be to being exploring the data. In the next unit, we'll calculate simple, descriptive statistics and create exploratory visualizations using the data we prepared in this this unit.

## Resources and Further Reading

- [GeoPandas Documentation](http://geopandas.org/)
- [*Data Cleaning: Problems and Current Approaches* by Rahm and Do](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.98.8661&rep=rep1&type=pdf)
- [*Data Mining: Concepts and Techniques* by Han, Pei, and Kamber, Section 3.2: Data Cleansing (Safari Books)](http://proquest.safaribooksonline.com.cscc.ohionet.org/book/databases/data-warehouses/9780123814791/3dot-data-preprocessing/32_data_cleaning?uicode=ohlink)
- [*Python Data Science Handbook* by VanderPlas, Chapter 3: Data Manipulation with Pandas](https://jakevdp.github.io/PythonDataScienceHandbook/03.00-introduction-to-pandas.html)
- [*Python for Data Analysis* by Wes McKinney, Chapter 7: Data Cleaning and Preparation (Safari Books)](http://proquest.safaribooksonline.com.cscc.ohionet.org/book/programming/python/9781491957653/data-cleaning-and-preparation/data_preparation_html?uicode=ohlink)

## Exercises

Use this notebook to compete each exercise below.  Add cells as necessary. 

1. Road data, maintained by the State of Ohio, for Franklin County is available in `./data/02-roads.csv` with a description of columns in `./data/02-roads-description.csv`.  Load the data and perform the following tasks.

    - Filter the data to include only roads with speed limits of 55 mph or greater
    - Drop any columns missing data for every row in the filtered data 
    - Drop any rows missing data in the filtered data
    - Display the first few rows of filtered data
    
2. In the previous unit, we loaded data from two tables by performing a `JOIN` as part of a SQL Query using the following code.

    ```python
    from sqlalchemy import create_engine
    engine = create_engine('sqlite:///data/01-chinook.sqlite')

    query = """
    SELECT * 
    FROM Invoice INNER JOIN Customer 
    ON Invoice.CustomerId = Customer.CustomerId 
    ORDER BY Invoice.InvoiceID 
    """

    invoice_customer = pd.read_sql_query(query, engine)
    invoice_customer.head(5)
    ```
    Use pandas to retrieve the data from the `Invoice` and `Customer` tables as two separate DataFrames then use `merge()` to create a new DataFrame that joins the data.  Compare the result of using *merge()* to using `JOIN` in the SQL Query. Display the first few rows of the merged DataFrame.